In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 25, "lift_capacity": 4})
cfg['Lift']['num_chairs'] = 35

manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60*4)

# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

# sample an agent 
agent = manager.sample_agent()
INDEX_TIME = agent.get_activity_log_df()['time'].sample().iloc[0]
# this is the agent state @ time as self-determined (based on own logs, self.activity_log via self.get_latest_event)
state_gotten = agent.get_state(INDEX_TIME) 
agent

Agent(16) psychotic-fail 364535ed-ec10-499e-91c8-349083e2ab41

# Scratchpad (real)

In [6]:
raise Exception()

Exception: 

In [24]:
eid = '1dbbb0b5-eb57-49a0-a02c-501dffeb4b38'
print(agent.traceback_experience(eid))

# Experience Log 

*** 

A simulation of a ski lift queue has been ran. the lift has been operated from 
time `t` to time `t_{n}`. a time, sample_time=""

below are some parameters related to this simulation: 

exp_id: 1dbbb0b5-eb57-49a0-a02c-501dffeb4b38
agent_uuid: 791b38c3-889b-4962-a27b-d09085d7e4e4
return_event_uuid: 4719ac59-b078-4989-96a9-b018ddd2904c
simulation uuid: 


***


time_spent_in_queue: 0
time_spent_ride_lift: 6.960459961681367
time_spent_traverse_down_mountain: 6.361611653869799

***

# Full Dict

{
    "exp_id": "1dbbb0b5-eb57-49a0-a02c-501dffeb4b38",
    "return_event_uuid": "4719ac59-b078-4989-96a9-b018ddd2904c",
    "agent_uuid": "791b38c3-889b-4962-a27b-d09085d7e4e4",
    "agent_uuid_codename": "broken-phrase",
    "time_spent_ride_lift": 6.960459961681367,
    "time_spent_traverse_down_mountain": 6.361611653869799,
    "time_spent_in_queue": 0,
    "time_spent_total": 13.322071615551167
}

# Agent Log

{
    "ab92341b-c15c-4831-a9db-678d5ec04062": {
        "re

In [22]:
agent.experience_rideloop.log

{datetime.datetime(2025, 3, 12, 9, 7): {'exp_id': '758320f7-60bb-4d64-9639-0ef401db5fb7',
  'return_event_uuid': '096d9723-a078-4b0a-ae9c-8a42bf78b560',
  'agent_uuid': '791b38c3-889b-4962-a27b-d09085d7e4e4',
  'agent_uuid_codename': 'broken-phrase',
  'time_spent_ride_lift': 7.501415898520162,
  'time_spent_traverse_down_mountain': 5.745277375466008,
  'time_spent_in_queue': 0,
  'time_spent_total': 13.246693273986171},
 datetime.datetime(2025, 3, 12, 9, 17): {'exp_id': '45a319b7-952f-49af-9510-ceaffbe9a9ff',
  'return_event_uuid': '73a0fe70-c04a-450b-9313-06fdcfb0638c',
  'agent_uuid': '791b38c3-889b-4962-a27b-d09085d7e4e4',
  'agent_uuid_codename': 'broken-phrase',
  'time_spent_ride_lift': 5.783632339366831,
  'time_spent_traverse_down_mountain': 3.9078001476492736,
  'time_spent_in_queue': 0,
  'time_spent_total': 9.691432487016105},
 datetime.datetime(2025, 3, 12, 9, 29): {'exp_id': 'dd3fc0d3-5866-44f1-bf0c-24c4e2dad640',
  'return_event_uuid': '5b81d735-f50d-4faa-89f1-c7bc860334

# Scratchpad

In [ ]:
raise Exception()

# Agent State Identification (ride loop exp)

for the ride loop agent experience, the total time is broken down into riding the lift, traversing down the mountain and waiting in the queue, i.e. "time_spent_total": ride + traverse + queue below:

sample a random time within the time range of the simulation:

In [ ]:
# sample a random timestamp within the agent log timeframe
INDEX_TIME = pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [ ]:
s = inspect.getsource(AgentRideLoopExperience)
lines = s.split('\n')
idx = [i for i,x in enumerate(lines) if x.strip() == '"""'][-1]
print('\n'.join(lines[idx+1:]))

...and we can see in the source code above how `time_spent_total` is constructed. 

In [ ]:
from zero_liftsim.agent_state_id import infer_agent_states

# func 
r = infer_agent_states([agent], INDEX_TIME)

i = agent.agent_uuid
msg = 'func infer state should return agent uuid'
assert i in r, msg
print(r)

# Sample and Manually Review Examples

We need to evaluate how well the simulation is representing the actual constrained behavior that we see in the real world. Simply manually reviewing some examples here helps with that need. 

In [ ]:
def sample_time(log):
    return pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

In [ ]:
time = sample_time(log)
agent, l, e = sample_agent(manager, log, exp)
agent.get_state(time)
agent.current_state

x = agent.traceback_experience(e['exp_id'])
print(x)